In [1]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import json
%matplotlib inline

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 300

In [2]:
import os
%pwd

u'/Users/Driz/data_sci_wk3'

In [15]:
data = pd.read_csv('../DSI-CHI-1/train_data.tsv', sep='\t', na_values='?')

# Extract the title and body from the boilerplate JSON text
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', '')).fillna('')
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', '')).fillna('')

In [17]:
data[['title', 'label']].head()

,title,label
0,"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries",0
1,"The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races",1
2,Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health,1
3,10 Foolproof Tips for Better Sleep,1
4,The 50 Coolest Jerseys You Didn t Know Existed coolest jerseys you haven't seen,0


In [69]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(
    binary=True, # creates binary features (1,0)
    stop_words='english', #ignores common words
    max_features=50) #grabs top 50 common words

In [42]:
# builds matrix, row per website/column per word (using all filtered words)
vector_matrix = v.fit_transform(data.title).todense()
vector_matrix = pd.DataFrame(X, columns=v.get_feature_names())
vector_matrix.head()

,10,2011,allrecipes,apple,baked,best,blog,butter,cake,cheese,chicken,chocolate,com,cookies,cooking,cream,cupcakes,day,easy,fashion,food,foods,free,health,healthy,home,homemade,illustrated,insidershealth,kitchen,life,make,new,news,peanut,photos,pie,recipe,recipes,si,sports,style,sweet,swimsuit,technology,time,tips,video,ways,world
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
vector_matrix.sum().sort_values(ascending=False)

recipe            580
com               572
recipes           397
food              301
chocolate         260
sports            257
video             227
news              218
best              204
fashion           179
10                172
health            169
new               162
chicken           160
kitchen           159
cake              157
blog              155
make              154
cheese            128
world             119
cookies           118
cooking           116
life              114
swimsuit          111
illustrated       109
butter            108
cream             105
technology         98
easy               95
cupcakes           95
healthy            95
photos             92
allrecipes         90
home               89
si                 89
style              86
tips               85
baked              84
peanut             82
ways               81
insidershealth     79
free               78
foods              78
pie                76
sweet              75
time      

In [44]:
from sklearn.linear_model import LogisticRegression

In [48]:
def examine_coefficients(model, vector_matrix):
    vector_matrix = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : vector_matrix.columns}
    ).sort_values(by='Coefficient')
    return vector_matrix[vector_matrix.Coefficient !=0]

In [52]:
X = vector_matrix[[
        'cheese',
        'technology',
        'free'
    ]]
y = data.label


model = LogisticRegression() 

model.fit(X, y) # This fits the model to learn the coefficients
examine_coefficients(model, X)

,Coefficient,Feature
1,-2.104370,technology
2,-0.035587,free
0,2.289383,cheese


In [53]:
model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
import numpy as np
from sklearn import datasets
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV

In [64]:
X = vector_matrix
y = data.label


model = LogisticRegression() 

model.fit(X, y) # This fits the model to learn the coefficients
examine_coefficients(model, X)

,Coefficient,Feature
19,-1.704297,fashion
44,-1.702463,technology
33,-1.208896,news
47,-1.117272,video
40,-1.002996,sports
32,-0.719687,new
41,-0.479148,style
35,-0.465759,photos
45,-0.449675,time
1,-0.444006,2011


WALK THROUGH TOGETHER IN CLASS

In [67]:
# we want to pull keywords from more than just columns
data['body'].head()

0    A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose Califor...
1    And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a tr...
2    Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent heart disease and cancer Next Papayas With 250 percent of the RDA of vitamin C a papaya can help kick ...
3    There was a period in my life when I had a lot of problems with sleep It took me

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
data = CountVectorizer(
    binary=True, # creates binary features (1,0)
    stop_words='english', #ignores common words
    max_features=50) #grabs top 50 common words

In [73]:
body_talk = count_vectorizer.fit_transform(data.body).todense()
body_talk

NameError: name 'count_vectorizer' is not defined

In [ ]:
body_talk = pd.DataFrame(body_talk, columns=count_vectorizer.get_feature_names())
body_talk.head()
body_talk.sum().sort_values(ascending=False)
# print type(body_talk.sum())

In [74]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# join the two
my_xs = pf.DataFrame(X).join(pd.DataFrame(body_talk)

In [75]:
pd.DataFrame.join?

In [86]:
my_xs = pd.DataFrame(X).join(pd.DataFrame(body_talk), lsuffix='_title', rsuffix='_body')
my_xs.head()

NameError: name 'body_talk' is not defined

In [ ]:
# another way to join
print X.shape
print body_talk.shape
pd.concat([X, body_talk], axis=1)
len(pd.concat([X, body_talk], axis=1))

In [76]:
# penalty is L1 model (Lasso)
logistic_model_1 = LogisticRegression(penalty='11')

In [77]:
logistic_model_1.fit(my_xs, y)

NameError: name 'my_xs' is not defined

In [ ]:
logistic_model_1.score(my_xs, y)

In [ ]:
# here we made a new model object with a different C value
logistic_model_2 = LogisticRegression(penalty ='l1', C=0.1)
logistic_model_2

In [ ]:
logistic_model_2.fit(my_xs, y)

In [ ]:
logistic_model_2.score(my_xs, y)

GRID SEARCH TIME

In [ ]:
# Iterate over L1 and L2 penalty and 5 different C values

In [78]:
from sklearn.grid_search import GridSearchCV

In [79]:
parameters = {
    'penalty': ['l1','l2'],
    'C': [0.1, 1, 10,23.5,50]
}
model_i_am_testing = LogisticRegression()

In [82]:
grid_search = GridSearchCV(estimator=model_i_am_testing, 
                          param_grid=parameters,
                          verbose=10,
                          cv=6)

In [83]:
grid_search 

GridSearchCV(cv=6, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.1, 1, 10, 23.5, 50]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=10)

In [84]:
grid_search.fit(my_xs, y)

NameError: name 'my_xs' is not defined

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.best_estimator_.fit(my_xs, y)

In [ ]:
grid_search.best_estimator_.score(my_xs, y)

In [ ]:
from sklearn.metric import confusion_matrix, classification_report

In [ ]:
predictions = grid_search.best_estimator_.predict(my_xs)

In [ ]:
# a few lessons ago we put this into a dataframe (so it looks nice)
confusions_matrix(y, predictions)

In [ ]:
print classification_report(y, predictions)

In [ ]:
# visualizations for confusion matrix (looks pretty)
import seaborn as sns
sns.heatmap(confusion_matrix(y, predictions), annot=True)